In [2]:
import pandas as pd
import glob

## Functions and Variables

In [5]:
# Remove os avisos de dtype do pandas
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

In [3]:
def find_files(path, pattern):
    files = glob.glob(f'{path}/{pattern}')

    return files

In [9]:
def find_double_header(df):
    for coluna in df.columns:
        df.drop(df.loc[df[coluna] == coluna].index, inplace=True)

    return df

---

## ANSI Encoding
Some files from DATASUS have encoding set to ANSI, while the vast maiority have UTF-8
The cell below seeks to solve this error

In [ ]:
# Resolve erro de Codificação ANSI e combina arquivos do mesmo ano

for i in range(12, 23):
    files = find_files('C:\\Users\\ricmo\\OneDrive\\Documentos\\SIM - Agregação\\SIM\\DO', f'*\\*20{i}.csv', i)

    aux = pd.DataFrame()

    for item in files:
        try:
            df = pd.read_csv(item)
        except UnicodeDecodeError:
            df = pd.read_csv(item, encoding='ANSI')

        aux = pd.concat([aux, df], axis=0)

    aux = aux.drop_duplicates()
    aux.to_csv(f'C:\\Users\\ricmo\\OneDrive\\Documentos\\SIM - Agregação\\1. Agregação + ANSI\\DO20{i}.csv', index=False)


---

## Double Header Error
Some files also display the first N lines copied at the end of it.
The cell below solves this issue

In [ ]:
for i in range(12, 23):
    files = find_files('C:\\Users\\ricmo\\OneDrive\\Documentos\\SIM - Agregação\\1. Agregação + ANSI', f'*20{i}.csv', i)

    for item in files:
        df = pd.read_csv(item)
        df = find_double_header(df)

        df = df.drop_duplicates()
        df.to_csv(f'C:\\Users\\ricmo\\OneDrive\\Documentos\\SIM - Agregação\\2. Double Header Corrected\\DO20{i}.csv',
                index=False)

---

## Data Filtering
The database used features all the deaths that occurred in a year. 
We only need the ones from indivuduals with less younger than 1 year old

In [ ]:
# filtragem de dados
for i in range(12, 23):
    files = find_files('D:\\_repositories\\Aggregation\\Results\\SIMDO-DHC', f'*20{i}.csv', i)

    for item in files:
        df = pd.read_csv(item)
        
        df['IDADE'] = df['IDADE'].astype(str)
        aux = df[df['IDADE'].apply(lambda x: x[0] in ['1', '2', '3'])]

        print(f'Tamanho do Banco SIM DO do ano de 20{i}: {len(df)}, tamanho filtrado: {len(aux)} - Diferenca: {len(df) - len(aux)}')

        aux.to_csv(f'D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO20{i}.csv', index=False)

In [12]:
# Corrigindo DOFET
for i in range(12, 23):
    files = find_files('D:\\_repositories\\Databases\\SIM\\DOFET', f'*{i}.csv', i)

    for item in files:
        try:
            df = pd.read_csv(item)
        except UnicodeDecodeError:
            df = pd.read_csv(item, encoding='ANSI')

        df = find_double_header(df)

        for colunas in df.columns:
            try:
                df[colunas] = df[colunas].astype('Int64')
            except ValueError:
                continue
            except TypeError:
                continue

        df = df.drop_duplicates()
        df.to_csv(f'D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET{i}.csv', index=False)

['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET12.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET13.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET14.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\4252435703.py:9: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,51,53,54,55,56,57,58,59,61,68,69,73,74,75,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item, encoding='ANSI')


['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET15.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\4252435703.py:7: DtypeWarning: Columns (90,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET16.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\4252435703.py:7: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET17.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET18.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET19.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\4252435703.py:7: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET20.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET21.csv']
1
['D:\\_repositories\\Databases\\SIM\\DOFET\\DOFET22.csv']
1


In [16]:
for i in range(12, 23):
    do = find_files('D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado', f'*20{i}.csv', i)
    dofet = find_files('D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido', f'*{i}.csv', i)

    # aux = pd.DataFrame()
    for item1, item2 in zip(do, dofet):
        # print(f'{item1} {item2}')

        df = pd.read_csv(item1)
        aux = pd.read_csv(item2)

        aux = pd.concat([aux, df], axis=0)

        aux.to_csv(f'D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM20{i}.csv')

['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2012.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET12.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2013.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET13.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2014.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET14.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:9: DtypeWarning: Columns (20,80,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item1)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:10: DtypeWarning: Columns (23,51,53,90,92) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv(item2)


['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2015.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET15.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:9: DtypeWarning: Columns (20,80,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item1)
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:10: DtypeWarning: Columns (90,92) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv(item2)


['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2016.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET16.csv']
1


...

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:9: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item1)


['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2021.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET21.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:9: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item1)


['D:\\_repositories\\Aggregation\\Results\\SIMDO-Filtrado\\DO2022.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIMDOFET-Corrigido\\DOFET22.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\2613542773.py:9: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item1)


---

In [17]:
for i in range(12, 23):
    files = find_files('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado', f'*20{i}.csv', i)
    
    for item in files:
        df = pd.read_csv(item)
        

['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2012.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2013.csv']
1
['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2014.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1923534977.py:5: DtypeWarning: Columns (24,52,53,54,63,64,65,66,67,91,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2015.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1923534977.py:5: DtypeWarning: Columns (24,91,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2016.csv']
1


...

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1923534977.py:5: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2021.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1923534977.py:5: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


['D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2022.csv']
1


C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1923534977.py:5: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(item)


---

In [18]:
df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2022.csv')
df.info()

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1071666090.py:1: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2022.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58791 entries, 0 to 58790
Data columns (total 100 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  58791 non-null  int64  
 1   ORIGEM        58791 non-null  int64  
 2   CODMUNCART    0 non-null      float64
 3   NUMREGCART    0 non-null      float64
 4   CODCART       0 non-null      float64
 5   DTREGCART     0 non-null      float64
 6   TIPOBITO      58791 non-null  int64  
 7   DTOBITO       58791 non-null  int64  
 8   HORAOBITO     56580 non-null  float64
 9   NATURAL       54301 non-null  float64
 10  CODMUNNATU    53916 non-null  float64
 11  DTNASC        31233 non-null  float64
 12  IDADE         31397 non-null  float64
 13  SEXO          58791 non-null  int64  
 14  RACACOR       29491 non-null  float64
 15  ESTCIV        41 non-null     float64
 16  ESC           0 non-null      float64
 17  ESC2010       0 non-null      float64
 18  SERIESCFAL    0 non-null 

In [20]:
# E – estadual; R- regional; M- Municipal
# 1 - Estadual; 2 - Regional. 3 - Municipal

df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'R'] = 2
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'M'] = 3

df['TPNIVELINV'] = df['TPNIVELINV'].astype('Int64')

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'R'] = 2
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[

In [22]:
for colunas in df.columns:
    try:
        df[colunas] = df[colunas].astype('Int64')
    except ValueError:
        continue
    except TypeError:
        continue

In [23]:
df.to_csv('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\csv\\SIM2022.csv', index=False)
df.to_parquet('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\parquet\\SIM2022.parquet', index=False)

---

In [24]:
df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2021.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60219 entries, 0 to 60218
Data columns (total 100 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  60219 non-null  int64  
 1   ORIGEM        60219 non-null  int64  
 2   CODMUNCART    0 non-null      float64
 3   NUMREGCART    0 non-null      float64
 4   CODCART       0 non-null      float64
 5   DTREGCART     0 non-null      float64
 6   TIPOBITO      60219 non-null  int64  
 7   DTOBITO       60219 non-null  int64  
 8   HORAOBITO     57731 non-null  float64
 9   NATURAL       55304 non-null  float64
 10  CODMUNNATU    54848 non-null  float64
 11  DTNASC        30747 non-null  float64
 12  IDADE         30894 non-null  float64
 13  SEXO          60219 non-null  int64  
 14  RACACOR       28619 non-null  float64
 15  ESTCIV        45 non-null     float64
 16  ESC           0 non-null      float64
 17  ESC2010       0 non-null      float64
 18  SERIESCFAL    0 non-null 

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\407452425.py:1: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2021.csv')


In [25]:
# E – estadual; R- regional; M- Municipal
# 1 - Estadual; 2 - Regional. 3 - Municipal

df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'R'] = 2
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'M'] = 3

df['TPNIVELINV'] = df['TPNIVELINV'].astype('Int64')

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: SettingWithCo

In [26]:
for colunas in df.columns:
    try:
        df[colunas] = df[colunas].astype('Int64')
    except ValueError:
        continue
    except TypeError:
        continue

In [27]:
df.to_csv('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\csv\\SIM2021.csv', index=False)
df.to_parquet('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\parquet\\SIM2021.parquet', index=False)

---

In [28]:
df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2020.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59549 entries, 0 to 59548
Data columns (total 100 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  59549 non-null  int64  
 1   ORIGEM        59549 non-null  int64  
 2   CODMUNCART    0 non-null      float64
 3   NUMREGCART    0 non-null      float64
 4   CODCART       0 non-null      float64
 5   DTREGCART     0 non-null      float64
 6   TIPOBITO      59549 non-null  int64  
 7   DTOBITO       59549 non-null  int64  
 8   HORAOBITO     56771 non-null  float64
 9   NATURAL       54179 non-null  float64
 10  CODMUNNATU    53710 non-null  float64
 11  DTNASC        30375 non-null  float64
 12  IDADE         30556 non-null  float64
 13  SEXO          59549 non-null  int64  
 14  RACACOR       28192 non-null  float64
 15  ESTCIV        52 non-null     float64
 16  ESC           2 non-null      float64
 17  ESC2010       1 non-null      float64
 18  SERIESCFAL    0 non-null 

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\1517643345.py:1: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2020.csv')


In [29]:
# E – estadual; R- regional; M- Municipal
# 1 - Estadual; 2 - Regional. 3 - Municipal

df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'R'] = 2
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'M'] = 3

df['TPNIVELINV'] = df['TPNIVELINV'].astype('Int64')

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: SettingWithCo

In [30]:
for colunas in df.columns:
    try:
        df[colunas] = df[colunas].astype('Int64')
    except ValueError:
        continue
    except TypeError:
        continue

In [31]:
df.to_csv('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\csv\\SIM2020.csv', index=False)
df.to_parquet('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\parquet\\SIM2020.parquet', index=False)

---

In [32]:
df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2019.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63376 entries, 0 to 63375
Data columns (total 100 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  63376 non-null  int64  
 1   ORIGEM        63376 non-null  int64  
 2   CODMUNCART    0 non-null      float64
 3   NUMREGCART    0 non-null      float64
 4   CODCART       0 non-null      float64
 5   DTREGCART     0 non-null      float64
 6   TIPOBITO      63376 non-null  int64  
 7   DTOBITO       63376 non-null  int64  
 8   HORAOBITO     60399 non-null  float64
 9   NATURAL       57439 non-null  float64
 10  CODMUNNATU    57022 non-null  float64
 11  DTNASC        34075 non-null  float64
 12  IDADE         34271 non-null  float64
 13  SEXO          63376 non-null  int64  
 14  RACACOR       31791 non-null  float64
 15  ESTCIV        31 non-null     float64
 16  ESC           0 non-null      float64
 17  ESC2010       0 non-null      float64
 18  SERIESCFAL    0 non-null 

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\4181929245.py:1: DtypeWarning: Columns (91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\_repositories\\Aggregation\\Results\\SIM-Concatenado\\SIM2019.csv')


In [33]:
# E – estadual; R- regional; M- Municipal
# 1 - Estadual; 2 - Regional. 3 - Municipal

df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'R'] = 2
df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'M'] = 3

df['TPNIVELINV'] = df['TPNIVELINV'].astype('Int64')

C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['TPNIVELINV'].loc[df['TPNIVELINV'] == 'E'] = 1
C:\Users\ricmo\AppData\Local\Temp\ipykernel_16956\424150026.py:4: SettingWithCo

In [34]:
for colunas in df.columns:
    try:
        df[colunas] = df[colunas].astype('Int64')
    except ValueError:
        continue
    except TypeError:
        continue

In [35]:
df.to_csv('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\csv\\SIM2019.csv', index=False)
df.to_parquet('D:\\_repositories\\Aggregation\\Results\\SIM-DtypeCorrigido\\parquet\\SIM2019.parquet', index=False)